In [1]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook

In [2]:
import sys
import os

sys.path.append('../tools')

import multicrop_dataset
import architecture

In [3]:
import numpy as np
import matplotlib

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import sklearn

import torchvision.transforms as T
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks

matplotlib.use("nbAgg")

import data_utility

from itertools import groupby
from tqdm import tqdm

In [4]:
import matplotlib.pyplot as plt

In [5]:
data_dir = "../../../user_data/"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'

In [6]:
dir_list = os.listdir(data_dir+'rns_data')
patientIDs = [s for s in dir_list for type_string in ['HUP', 'RNS'] if type_string in s.upper()]

In [7]:
patientIDs

['HUP047',
 'HUP059',
 'HUP084',
 'HUP096',
 'HUP101',
 'HUP108',
 'HUP109',
 'HUP121',
 'HUP127',
 'HUP128',
 'HUP129',
 'HUP131',
 'HUP136',
 'HUP137',
 'HUP143',
 'HUP147',
 'HUP153',
 'HUP156',
 'HUP159',
 'HUP182',
 'HUP192',
 'HUP197',
 'HUP199',
 'HUP205',
 'RNS021',
 'RNS022',
 'RNS026',
 'RNS029']

In [8]:
# data_import = data_utility.read_files(path = data_dir+'rns_data', path_data = data_dir+'rns_raw_cache',patientIDs=patientIDs[10:], annotation_only = False, verbose=True)
# ids = list(data_import.keys())

In [9]:
# window_len = 1
# stride = 1
# concat_n = 2
# for id in tqdm(ids):
#     data_import[id].set_window_parameter(window_length=window_len, window_displacement=stride)
#     data_import[id].set_concatenation_parameter(concatenate_window_n=concat_n)
#     data_import[id].get_windowed_data(data_import[id].catalog["Event Start idx"],data_import[id].catalog["Event End idx"])
#     data_import[id].normalize_windowed_data()
#     _, concatenated_data = data_import[id].get_concatenated_data(data_import[id].windowed_data, arrange='channel_stack')
#     np.save(data_dir+'rns_cache/'+ id +'.npy',concatenated_data)

In [10]:
file_list = ['HUP159.npy']

In [11]:
from models.rns_dataloader import RNS_Raw
unlabeled_dataset = RNS_Raw(file_list, transform=True,astensor = False)

(0, 249, 20)


100%|██████████| 1/1 [00:00<00:00,  1.76it/s]

data loaded


In [18]:
from models.SwaV import SwaV
from lightly.data import SwaVCollateFunction
#
model = SwaV()

# model.load_from_checkpoint()

collate_fn = SwaVCollateFunction(gaussian_blur = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0, normalize={'mean':[0, 0, 0], 'std':[1, 1, 1]})

dataloader = torch.utils.data.DataLoader(
    unlabeled_dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='swav_loss',filename='rns_swav_159-{epoch:02d}-{swav_loss:.5f}', save_last=True, save_top_k=-1, dirpath=ckpt_folder_root + 'rns_swav_34')
csv_logger = pl_loggers.CSVLogger(log_folder_root, name="rns_swav_34")

trainer = pl.Trainer(logger=csv_logger, max_epochs=150, callbacks=[checkpoint_callback],accelerator='gpu', devices=1,precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(model=model, train_dataloaders=dataloader,ckpt_path=ckpt_folder_root+'rns_swav_34/rns_swav_159-epoch=109-swav_loss=3.12453.ckpt')

Restoring states from the checkpoint path at ../../../user_data/checkpoints/rns_swav_34/rns_swav_159-epoch=109-swav_loss=3.12453.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | backbone        | Sequential         | 21.3 M
1 | projection_head | SwaVProjectionHead | 328 K 
2 | prototypes      | SwaVPrototypes     | 66.0 K
3 | queues          | ModuleList         | 0     
4 | criterion       | SwaVLoss           | 0     
-------------------------------------------------------
21.7 M    Trainable params
0         Non-trainable params
21.7 M    Total params
43.359    Total estimated model params size (MB)
Restored all states from the checkpoint file at ../../../user_data/checkpoints/rns_swav_34/rns_swav_159-epoch=109-swav_loss=3.12453.ckpt


Training: 159it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=150` reached.
